# 5.A More Difficult Classification Problem

In [19]:
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

### Choose the necessary def from the mp1.py

In [20]:
def generate_a_drawing(figsize, U, V, noise=0.0):
    fig = plt.figure(figsize=(figsize,figsize))
    ax = plt.subplot(111)
    plt.axis('Off')
    ax.set_xlim(0,figsize)
    ax.set_ylim(0,figsize)
    ax.fill(U, V, "k")
    fig.canvas.draw()
    imdata = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)[::3].astype(np.float32)
    imdata = imdata + noise * np.random.random(imdata.size)
    plt.close(fig)
    return imdata

def generate_a_rectangle(noise=0.0, free_location=False):
    figsize = 1.0    
    U = np.zeros(4)
    V = np.zeros(4)
    if free_location:
        corners = np.random.random(4)
        top = max(corners[0], corners[1])
        bottom = min(corners[0], corners[1])
        left = min(corners[2], corners[3])
        right = max(corners[2], corners[3])
    else:
        side = (0.3 + 0.7 * np.random.random()) * figsize
        top = figsize/2 + side/2
        bottom = figsize/2 - side/2
        left = bottom
        right = top
    U[0] = U[1] = top
    U[2] = U[3] = bottom
    V[0] = V[3] = left
    V[1] = V[2] = right
    return generate_a_drawing(figsize, U, V, noise)

def generate_a_disk(noise=0.0, free_location=False):
    figsize = 1.0
    if free_location:
        center = np.random.random(2)
    else:
        center = (figsize/2, figsize/2)
    radius = (0.3 + 0.7 * np.random.random()) * figsize/2
    N = 50
    U = np.zeros(N)
    V = np.zeros(N)
    i = 0
    for t in np.linspace(0, 2*np.pi, N):
        U[i] = center[0] + np.cos(t) * radius
        V[i] = center[1] + np.sin(t) * radius
        i = i + 1
    return generate_a_drawing(figsize, U, V, noise)

def generate_a_triangle(noise=0.0, free_location=False):
    figsize = 1.0
    if free_location:
        U = np.random.random(3)
        V = np.random.random(3)
    else:
        size = (0.3 + 0.7 * np.random.random())*figsize/2
        middle = figsize/2
        U = (middle, middle+size, middle-size)
        V = (middle+size, middle-size, middle-size)
    imdata = generate_a_drawing(figsize, U, V, noise)
    return [imdata, [U[0], V[0], U[1], V[1], U[2], V[2]]]

def generate_dataset_classification(nb_samples, noise=0.0, free_location=False):
    # Getting im_size:
    im_size = generate_a_rectangle().shape[0]
    X = np.zeros([nb_samples,im_size])
    Y = np.zeros(nb_samples)
    print('Creating data:')
    for i in range(nb_samples):
        if i % 10 == 0:
            print(i, end=' ')
        category = np.random.randint(3)
        if category == 0:
            X[i] = generate_a_rectangle(noise, free_location)
        elif category == 1: 
            X[i] = generate_a_disk(noise, free_location)
        else:
            [X[i], V] = generate_a_triangle(noise, free_location)
        Y[i] = category
    X = (X + noise) / (255 + 2 * noise)
    return [X, Y]

def generate_test_set_classification():
    np.random.seed(42)
    [X_test, Y_test] = generate_dataset_classification(300, 20, True)
    return [X_test, Y_test]

### Generate new training and testing set 

In [21]:
[X_train, Y_train] = generate_dataset_classification(300, 20, True)
[X_test, Y_test] = generate_test_set_classification()

Creating data:
0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 Creating data:
0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 

In [22]:
X_train = X_train.reshape(len(X_train), 72, 72, 1)
X_test = X_test.reshape(len(X_test), 72, 72, 1)
y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

### Build and Compile the model

In [23]:
model = Sequential()
model.add(Conv2D(kernel_size=(5, 5), padding="same",input_shape=(72, 72, 1),filters=16, activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

In [24]:
model.add(Conv2D(32, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

In [25]:
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training the model

In [27]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Train on 300 samples, validate on 300 samples
Epoch 1/10
300/300 [==============================] - 21s 69ms/step - loss: 1.1053 - acc: 0.3733 - val_loss: 1.0795 - val_acc: 0.3300
Epoch 2/10
300/300 [==============================] - 21s 71ms/step - loss: 0.9745 - acc: 0.5667 - val_loss: 0.8937 - val_acc: 0.5933
Epoch 3/10
300/300 [==============================] - 21s 71ms/step - loss: 0.7259 - acc: 0.7000 - val_loss: 0.7476 - val_acc: 0.6733
Epoch 4/10
300/300 [==============================] - 21s 69ms/step - loss: 0.5612 - acc: 0.7267 - val_loss: 0.7628 - val_acc: 0.6733
Epoch 5/10
300/300 [==============================] - 21s 70ms/step - loss: 0.4124 - acc: 0.8467 - val_loss: 0.7061 - val_acc: 0.7033
Epoch 6/10
300/300 [==============================] - 21s 72ms/step - loss: 0.2652 - acc: 0.9033 - val_loss: 0.8573 - val_acc: 0.6700
Epoch 7/10
300/300 [==============================] - 21s 69ms/step - loss: 0.1954 - acc: 0.9300 - val_loss: 0.8468 - val_acc: 0.7200
Epoch 8/10
300/3

#### We can see that finally in the training set we will get the value of loss function of 0.0405 and the accuracy rate of 0.9967.

### Valuate my classifier on this test set

In [30]:
model.evaluate(X_test, y_test)

300/300 [==============================] - 4s 15ms/step


[0.7711150662104289, 0.7933333325386047]

#### The value of loss function in testing set is 0.771 and accuracy rate is 0.79.